as we had added the rnn , so for understanding why memory helps, i had break down this code in three major step which are : 1) replacing bigram with rnn, 2)comparing the output and 3) stress test memory

In [18]:
#making raw dataset
text = """ In the beginning the universe was created. This has made a lot of people very angry and been widely regarded as bad move"""
print(text)

 In the beginning the universe was created. This has made a lot of people very angry and been widely regarded as bad move


In [19]:
# tokenizations
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("Charatcters:",chars)
print("Vocab Size:",vocab_size)

Charatcters: [' ', '.', 'I', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']
Vocab Size: 25


In [20]:
# build mapping
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s:[stoi[c] for c in s]
decode = lambda l:"".join([itos[i] for i in l])

print(encode("the"))
print(decode(encode("the")))

[20, 11, 8]
the


In [21]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data[:20])
print("Total tokens:",len(data))

tensor([ 0,  2, 15,  0, 20, 11,  8,  0,  5,  8, 10, 12, 15, 15, 12, 15, 10,  0,
        20, 11])
Total tokens: 121


In [22]:
block_size = 8 # context  lenghth
batch_size = 4

def get_batch():
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size]for i in ix])
  y = torch.stack([data[i+1:i+block_size+1]for i in ix])
  return x,y

x,y = get_batch()
print(x)
print(y)

tensor([[16,  9,  0, 17,  8, 16, 17, 13],
        [ 8, 18, 24,  0,  4, 15, 10, 18],
        [ 2, 15,  0, 20, 11,  8,  0,  5],
        [10,  4, 18,  7,  8,  7,  0,  4]])
tensor([[ 9,  0, 17,  8, 16, 17, 13,  8],
        [18, 24,  0,  4, 15, 10, 18, 24],
        [15,  0, 20, 11,  8,  0,  5,  8],
        [ 4, 18,  7,  8,  7,  0,  4, 19]])


In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [24]:
#RNN MODEL
class RNNLangaugeModel(nn.Module):
  def __init__ (self,vocab_size,embed_size=32, hidden_size=64):
    super().__init__()
    self.embed = nn.Embedding(vocab_size,embed_size)
    self.rnn = nn.RNN(embed_size,hidden_size,batch_first=True)
    self.fc = nn.Linear(hidden_size,vocab_size)

  def forward(self, x, targets=None):
    x =self.embed(x)
    out, _ = self.rnn(x)
    logits = self.fc(out)

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)

    return logits,loss



In [25]:
#train
model = RNNLangaugeModel(vocab_size=100)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

for step in range(5000):
  xb, yb = get_batch()
  logits, loss = model(xb,yb)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if step % 500 == 0:
    print(f"Step{step}| Loss{loss.item():.4f}")

Step0| Loss4.6678
Step500| Loss0.4885
Step1000| Loss0.2386
Step1500| Loss0.3234
Step2000| Loss0.3738
Step2500| Loss0.3523
Step3000| Loss0.3009
Step3500| Loss0.2663
Step4000| Loss0.1725
Step4500| Loss0.2671


In [26]:
#genrat text
def generate_rnn(model,start_char,max_new_tokens = 100):
  idx = torch.tensor([[stoi[start_char]]])

  for _ in range(max_new_tokens):
    logits, _ = model(idx)
    logits =logits[:,-1,:]
    probs = F.softmax (logits, dim=-1)
    next_idx = torch.multinomial(probs,1)
    idx = torch.cat([idx,next_idx],dim=1)

  return decode(idx[0].tolist())
print(generate_rnn(model,'I'))



In the beginning the universe was created. This has made a lot of people very angry This has made a l


the core concept i learn from this lesson is that , Memory helps, but squential processing is a dead end at scale